<a href="https://colab.research.google.com/github/cyc1974/BPI2019/blob/main/pytorch_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import numpy as np
wine_path='winequality-white.csv'
wineq_np=np.loadtxt(wine_path,dtype=np.float32,delimiter=";",skiprows=1)
wineq_np

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [4]:
col_list=next(csv.reader(open(wine_path),delimiter=';')) #取得標題列
wineq_np.shape,'/n',col_list

((4898, 12),
 '/n',
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [6]:
import torch
wineq=torch.from_numpy(wineq_np)
wineq.shape,wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [8]:
data=wineq[:,:-1] #刪除品質分數行
data.shape

torch.Size([4898, 11])

In [9]:
target=wineq[:,-1] #取出品質分數行
target.shape

torch.Size([4898])

In [11]:
target=wineq[:,-1].long() #轉為整數
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In [17]:
#建立shape為[4898，10」,值均為0的tensor,用來存放編碼結果
t_onehot=torch.zeros(target.shape[0],10)
a=t_onehot.scatter_(1,target.unsqueeze(1),1.0)
a,' ',a.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]), ' ', torch.Size([4898, 10]))

In [19]:
#添加一個軸至第1軸
t_unsqueezed=target.unsqueeze(1)
t_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [23]:
#計算每行的平均值
data_mean=torch.mean(data,dim=0) #dim=0代表延著第0軸做計算

data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [24]:
#計算標準差
data_var=torch.var(data,dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [26]:
#標準化
data_normalized=(data-data_mean)/torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [29]:
#找出品質差的酒數量（先定義品質小於3為品質差）
bad_index=target<=3
bad_index.shape,bad_index.dtype,bad_index.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [40]:
#利用索引找出符合目標的項目
bad_data=data[bad_index]
bad_data

tensor([[8.5000e+00, 2.6000e-01, 2.1000e-01, 1.6200e+01, 7.4000e-02, 4.1000e+01,
         1.9700e+02, 9.9800e-01, 3.0200e+00, 5.0000e-01, 9.8000e+00],
        [5.8000e+00, 2.4000e-01, 4.4000e-01, 3.5000e+00, 2.9000e-02, 5.0000e+00,
         1.0900e+02, 9.9130e-01, 3.5300e+00, 4.3000e-01, 1.1700e+01],
        [9.1000e+00, 5.9000e-01, 3.8000e-01, 1.6000e+00, 6.6000e-02, 3.4000e+01,
         1.8200e+02, 9.9680e-01, 3.2300e+00, 3.8000e-01, 8.5000e+00],
        [7.1000e+00, 3.2000e-01, 3.2000e-01, 1.1000e+01, 3.8000e-02, 1.6000e+01,
         6.6000e+01, 9.9370e-01, 3.2400e+00, 4.0000e-01, 1.1500e+01],
        [6.9000e+00, 3.9000e-01, 4.0000e-01, 4.6000e+00, 2.2000e-02, 5.0000e+00,
         1.9000e+01, 9.9150e-01, 3.3100e+00, 3.7000e-01, 1.2600e+01],
        [1.0300e+01, 1.7000e-01, 4.7000e-01, 1.4000e+00, 3.7000e-02, 5.0000e+00,
         3.3000e+01, 9.9390e-01, 2.8900e+00, 2.8000e-01, 9.6000e+00],
        [7.9000e+00, 6.4000e-01, 4.6000e-01, 1.0600e+01, 2.4400e-01, 3.3000e+01,
         2.27

In [35]:
a=torch.arange(10)
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [37]:
b=a>=5
b

tensor([False, False, False, False, False,  True,  True,  True,  True,  True])

In [41]:
a[b]

tensor([5, 6, 7, 8, 9])

In [49]:
#將品質分為三類
bad_data=data[target<=3]
mid_data=data[(target>3) & (target<7)]
good_data=data[target>7]
bad_mean=torch.mean(bad_data,dim=0)
mid_mean=torch.mean(mid_data,dim=0)
good_mean=torch.mean(good_data,dim=0)
for i ,args in enumerate(zip(col_list,bad_mean,mid_mean,good_mean)):
  print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i,*args))


 0 fixed acidity          7.60   6.89   6.68
 1 volatile acidity       0.33   0.28   0.28
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.63
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  36.63
 6 total sulfur dioxide 170.60 141.83 125.88
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.49
10 alcohol               10.34  10.26  11.65


In [59]:
#設定閥值
total_sulfur_threshold=141.83 #A
total_sulfur_data=data[:,6]  #B
predicated_indexs=torch.lt(total_sulfur_data,total_sulfur_threshold)  #用lt挑出A資料中值小於B的索引
predicated_indexs.shape,predicated_indexs.dtype,predicated_indexs.sum()


(torch.Size([4898]), torch.bool, tensor(2727))

In [61]:
#驗證合理性，先找出target>5
actual_indexs=target>5
actual_indexs.shape,actual_indexs.dtype,actual_indexs.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [63]:
#找出actual與predicated皆為TRUE的項目索引，計算結果吻合的數量
n_matches=torch.sum(actual_indexs & predicated_indexs).item()
n_p=torch.sum(predicated_indexs).item()
n_a=torch.sum(actual_indexs).item()
n_p,n_a
n_matches,n_matches/n_p,n_matches/n_a

(2018, 0.74000733406674, 0.6193984039287906)